# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **FPL player score** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, combine predictions with static data
3. Store predictions in a monitoring feature group

## <span style='color:#ff5f27'> 📝 Imports

In [216]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from functions import util
import os
from importlib import reload
import requests

In [217]:
reload(util)

print(util.get_gameweek_from_date("2025-1-4"))

20


## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [218]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
fs = project.get_feature_store()

2025-01-07 10:28:02,563 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 10:28:02,568 INFO: Initializing external client
2025-01-07 10:28:02,569 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 10:28:03,987 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159321


## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [219]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="player_score_xgboost_model",
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2025-01-07 10:28:05,674 WARNING: VersionWarning: No version provided for getting model `player_score_xgboost_model`, defaulting to `1`.



In [220]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='1.5633903E0', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'floa...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [221]:
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
general_info = requests.get(bootstrap_url).json()

In [222]:
try:
    monitor_fg = fs.get_feature_group(
            name='fpl_predictions',
            version=1,
        )
    prev_gameweek_nr = monitor_fg.select(["gameweek"]).read().max()
except:
    print("Feature group not found, use player features instead")
    player_fg = fs.get_feature_group("player_features")
    prev_gameweek_nr = player_fg.select(["gameweek"]).read().max()

Feature group not found, use player features instead
2025-01-07 10:28:08,689 WARNING: VersionWarning: No version provided for getting feature group `player_features`, defaulting to `1`.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.75s) 


In [226]:
prev_gameweek = next(
    (gw for gw in general_info['events'] if gw['id'] == int(prev_gameweek_nr)), None
)

prev_finished = prev_gameweek['finished']
prev_finished

2025-01-07 10:28:53,286 WARNING: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

2025-01-07 10:28:53,288 WARNING: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



18

## <span style="color:#ff5f27;">✨ Get Player Score Features From Previous Gameweek </span>



In [229]:
if prev_finished:
    players_fg = fs.get_feature_group(
        name='player_features',
        version=1,
    )
    players = players_fg.filter(players_fg.gameweek == int(prev_gameweek_nr)).read()
    print("get stats for finished gameweek")

2025-01-07 10:31:12,429 WARNING: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.45s) 
get stats for finished gameweek


### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [230]:
inputs = ['prev_minutes', 'prev_goals_scored', 'prev_assists',
         'prev_clean_sheets', 'prev_goals_conceded', 'prev_own_goals',
         'prev_penalties_saved', 'prev_penalties_missed', 'prev_yellow_cards',
         'prev_red_cards', 'prev_saves', 'prev_bonus',
         'prev_bps', 'prev_influence', 'prev_creativity',
         'prev_threat', 'prev_ict_index', 'prev_starts',
         'prev_expected_goals', 'prev_expected_assists', 'prev_expected_goal_involvements',
         'prev_expected_goals_conceded', 'prev_in_dreamteam', 'prev_total_points',
         'prev_selected', 'prev_transfers_balance', 'prev_value', 'prev_was_home']

#for i in range(0, len(input_data)):
#    output_data.at[i, 'predicted_score'] = retrieved_xgboost_model.predict(input_data.loc[i, inputs].values.reshape(1, -1))[0]

if prev_finished:
    players['predicted_score'] = retrieved_xgboost_model.predict(players[inputs]).astype("float64")
    print("made predictions")

made predictions


In [231]:
if prev_finished:
    players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               484 non-null    int64  
 1   points                           484 non-null    int64  
 2   gameweek                         484 non-null    int64  
 3   prev_minutes                     484 non-null    float64
 4   prev_goals_scored                484 non-null    float64
 5   prev_assists                     484 non-null    float64
 6   prev_clean_sheets                484 non-null    float64
 7   prev_goals_conceded              484 non-null    float64
 8   prev_own_goals                   484 non-null    float64
 9   prev_penalties_saved             484 non-null    float64
 10  prev_penalties_missed            484 non-null    float64
 11  prev_yellow_cards                484 non-null    float64
 12  prev_red_cards        

### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

### Create Score Graph
Draw a graph of the predictions with all the players as a PNG and save it to the github

In [232]:
if prev_finished:
    # Get or create feature group
    monitor_fg = fs.get_or_create_feature_group(
        name='fpl_predictions',
        description='FPL player score prediction monitoring',
        version=1,
        primary_key=['id', 'gameweek']
    )
    print("Got the monitor fg")

Got the monitor fg


In [233]:
if prev_finished:
    print("Setting the output data")
    # Only keep the relevant columns
    output_data = players[['id', 'points', 'gameweek', 'predicted_score']]

Setting the output data


In [ ]:
if prev_finished:
    # Add static data
    for i in range(0, len(output_data)):
        player_id = output_data.loc[i, 'id']

        player_data = util.get_player_info(player_id,general_info)
        for key in player_data:
            output_data.at[i, key] = player_data[key]
    print("Added static data")

In [ ]:
if prev_finished:
    monitor_fg.insert(output_data)
    print("Inserted the data")

In [189]:
if prev_finished:
    monitor_fg.update_feature_description("id", "Player ID")
    monitor_fg.update_feature_description("points", "Total points of the player in the gameweek (label)")
    monitor_fg.update_feature_description("gameweek", "Gameweek")
    monitor_fg.update_feature_description("predicted_score", "Predicted score of the player in the gameweek")
    monitor_fg.update_feature_description("first_name", "Player first name")
    monitor_fg.update_feature_description("second_name", "Player surname")
    monitor_fg.update_feature_description("team", "Player team")
    monitor_fg.update_feature_description("position", "Player position")
    print("Updated the descriptions")

---